### RNN을 이용하여 영화 추천
### 참고 자료 : 텐서플로우 공식 홈페이지 가이드 https://www.tensorflow.org/recommenders/examples/sequential_retrieval?hl=ko

In [2]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

### Step.0 라이브러리 준비

In [3]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

### Step.1 데이터 준비 (MovieLens 1M data contains ratings.dat )
- 평가 데이터와 영화 데이터를 2개의 데이터를 이용
- 평점 2이상인 데이터만 이용
- 이전 사용자 활동 10개를 예측 컨텍스트로 사용

In [4]:
!wget -nc https://raw.githubusercontent.com/tensorflow/examples/master/lite/examples/recommendation/ml/data/example_generation_movielens.py
!python -m example_generation_movielens  --data_dir=data/raw  --output_dir=data/examples  --min_timeline_length=3  --max_context_length=10  --max_context_movie_genre_length=10  --min_rating=2  --train_data_fraction=0.9  --build_vocabs=False

File ‘example_generation_movielens.py’ already there; not retrieving.

2023-05-30 14:57:53.845731: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
I0530 14:57:55.352593 139647104845632 example_generation_movielens.py:460] Downloading and extracting data.
I0530 14:57:55.530036 139647104845632 example_generation_movielens.py:406] Reading data to dataframes.
/content/example_generation_movielens.py:132: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings_df = pd.read_csv(
/content/example_generation_movielens.py:140: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='pytho

In [5]:
train_filename = "./data/examples/train_movielens_1m.tfrecord"
train = tf.data.TFRecordDataset(train_filename)

test_filename = "./data/examples/test_movielens_1m.tfrecord"
test = tf.data.TFRecordDataset(test_filename)

feature_description = {
    'context_movie_id': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(0, 10)),
    'context_movie_rating': tf.io.FixedLenFeature([10], tf.float32, default_value=np.repeat(0, 10)),
    'context_movie_year': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(1980, 10)),
    'context_movie_genre': tf.io.FixedLenFeature([10], tf.string, default_value=np.repeat("Drama", 10)),
    'label_movie_id': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
}

def _parse_function(example_proto):
  return tf.io.parse_single_example(example_proto, feature_description)

train_ds = train.map(_parse_function).map(lambda x: {
    "context_movie_id": tf.strings.as_string(x["context_movie_id"]),
    "label_movie_id": tf.strings.as_string(x["label_movie_id"])
})

test_ds = test.map(_parse_function).map(lambda x: {
    "context_movie_id": tf.strings.as_string(x["context_movie_id"]),
    "label_movie_id": tf.strings.as_string(x["label_movie_id"])
})

for x in train_ds.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'context_movie_id': array([b'372', b'141', b'11', b'2706', b'2699', b'2723', b'1441', b'785',
       b'236', b'339'], dtype=object),
 'label_movie_id': array([b'500'], dtype=object)}


0 : {
  features: {
    feature: {
      key  : "context_movie_id"
      value: { int64_list: { value: [ 1124, 2240, 3251, ..., 1268 ] } }
    }
    feature: {
      key  : "context_movie_rating"
      value: { float_list: {value: [ 3.0, 3.0, 4.0, ..., 3.0 ] } }
    }
    feature: {
      key  : "context_movie_year"
      value: { int64_list: { value: [ 1981, 1980, 1985, ..., 1990 ] } }
    }
    feature: {
      key  : "context_movie_genre"
      value: { bytes_list: { value: [ "Drama", "Drama", "Mystery", ..., "UNK" ] } }
    }
    feature: {
      key  : "label_movie_id"
      value: { int64_list: { value: [ 3252 ] }  }
    }
  }
}

컨텍스트 영화 ID 시퀀스, 레이블 영화 ID(다음 영화), 영화 연도, 등급 및 장르와 같은 컨텍스트 기능이 포함된 것을 볼 수 있습니다.
여기서는 컨텍스트 영화 ID 시퀀스와 레이블 영화 ID만 사용

이제 훈련/테스트 데이터 세트에는 과거 영화 ID의 시퀀스와 다음 영화 ID의 레이블만 포함됩니다. 예제 생성 단계에서 컨텍스트 특징의 길이를 10으로 지정했기 때문에 tf.Example 구문 분석 중에 특징의 모양으로 `[10]`을 사용한다는 점에 유의하세요.

모델 구축을 시작하기 전에 한 가지 더 필요한 것이 있는데, 바로 영화 ID에 대한 어휘입니다.

In [6]:
movies = tfds.load("movielens/1m-movies", split='train')
movies = movies.map(lambda x: x["movie_id"])
movie_ids = movies.batch(1_000)
unique_movie_ids = np.unique(np.concatenate(list(movie_ids)))

query_model은 사용자의 관심사나 행동 등을 기반으로 사용자 프로필을 나타내는 데 사용되고, candidate_model은 아이템 자체를 나타내는 데 사용된다. 이 두 모델의 출력은 내적 또는 코사인 유사도 등의 방식으로 비교되어 사용자에게 가장 관련성 있는 아이템을 추천하는 데 사용된다.

In [7]:
embedding_dimension = 32

query_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup( 
      vocabulary=unique_movie_ids, mask_token=None),
    #-- 문자열을 정수 인덱스로 변환. 이 경우에는 영화 ID를 해당 ID의 고유 인덱스로 변환.
    tf.keras.layers.Embedding(len(unique_movie_ids) + 1, embedding_dimension), 
    #-- 이 레이어는 인덱스를 고정된 크기의 벡터로 변환. 여기서는 각 영화 ID를 32차원의 임베딩 벡터로 변환.
    tf.keras.layers.GRU(embedding_dimension),
    #-- 이 레이어는 순환 신경망(RNN)의 한 종류인 게이트 순환 유닛(GRU)입니다. GRU 레이어는 시퀀스의 각 요소에 대해 상태를 업데이트하고,
    #-- 이 경우에는 마지막 상태만 출력.
])
#--  각 영화 를 32차원 벡터로 변환하고, 이를 GRU 레이어에 전달하여 시퀀스의 전체 정보를 캡쳐하는 최종 상태를 얻습니다.

candidate_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_ids, mask_token=None),
  #-- 문자열을 정수 인덱스로 변환. 이 경우에는 영화 ID를 해당 ID의 고유 인덱스로 변환.
  tf.keras.layers.Embedding(len(unique_movie_ids) + 1, embedding_dimension)
  #-- 이 레이어는 인덱스를 고정된 크기의 벡터로 변환. 여기서는 각 영화 ID를 32차원의 임베딩 벡터로 변환.
])
#-- 단일 영화를 32차원 벡터로 변환.

TensorFlow Recommenders(TFRS)를 사용하여 모델을 정의하고, 손실 함수를 계산하는 방법을 설명

In [8]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(candidate_model)
)
#-- TFRS의 FactorizedTopK 메트릭을 정의한다. 
#-- 이 메트릭은 추천 시스템에서 자주 사용되며, 모델이 각 쿼리에 대해 가장 관련성 높은 후보를 올바르게 식별하는 능력을 측정한다. 
#-- 여기서는 영화를 배치로 만든 후 후보 모델을 통과시켜 각 영화에 대한 임베딩을 계산하고, 이 임베딩을 후보로 사용한다.

task = tfrs.tasks.Retrieval(
  metrics=metrics
)
#-- 위에서 정의한 메트릭을 사용하여 Retrieval 작업을 정의. 이 작업은 각 쿼리에 대해 가장 관련성 있는 후보를 검색하는 것이다.

class Model(tfrs.Model): #-- TFRS의 모델 클래스를 상속하여 사용자 모델을 정의
    #-- 
    def __init__(self, query_model, candidate_model):
        super().__init__()
        self._query_model = query_model
        self._candidate_model = candidate_model

        self._task = task

    def compute_loss(self, features, training=False):
        #-- compute_loss 메서드는 모델의 손실을 계산하는데 사용.
        watch_history = features["context_movie_id"]
        watch_next_label = features["label_movie_id"]
        #-- 감시 히스토리(watch_history)와 감시 레이블(watch_next_label)을 추출한다. 
        query_embedding = self._query_model(watch_history)       
        candidate_embedding = self._candidate_model(watch_next_label)
        #-- 그런 다음 쿼리 모델과 후보 모델을 사용하여 각각의 임베딩을 계산
        return self._task(query_embedding, candidate_embedding, compute_metrics=not training)
        #-- 쿼리 임베딩과 후보 임베딩에 대한 손실을 계산

### Step3. 모델 학습
위의 코드는 사용자 정의 모델을 컴파일하고 학습시키는 과정을 설명합니다.

In [9]:
model = Model(query_model, candidate_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
#-- Adagrad 최적화 알고리즘을 사용하며, 이 최적화 알고리즘의 학습률은 0.1로 설정
cached_train = train_ds.shuffle(10_000).batch(12800).cache()
cached_test = test_ds.batch(2560).cache()
#-- 학습 데이터셋(train_ds)과 테스트 데이터셋(test_ds)를 셔플링하고 배치 형태로 변환하며, 
#-- 이를 캐시에 저장한다. 캐싱은 데이터를 빠르게 로딩하기 위해 사용됩니다.

model.fit(cached_train, epochs=3)
#-- 모델 학습 

Epoch 1/3
67/67 [==============================] - 39s 519ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 108442.1734 - regularization_loss: 0.0000e+00 - total_loss: 108442.1734
Epoch 2/3
67/67 [==============================] - 5s 67ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 101193.4503 - regularization_loss: 0.0000e+00 - total_loss: 101193.4503
Epoch 3/3
67/67 [==============================] - 4s 66ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 

### Step.4 모델 평가

In [10]:
model.evaluate(cached_test, return_dict=True)

37/37 [==============================] - 19s 488ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0144 - factorized_top_k/top_5_categorical_accuracy: 0.0774 - factorized_top_k/top_10_categorical_accuracy: 0.1352 - factorized_top_k/top_50_categorical_accuracy: 0.3741 - factorized_top_k/top_100_categorical_accuracy: 0.5079 - loss: 15479.7087 - regularization_loss: 0.0000e+00 - total_loss: 15479.7087


{'factorized_top_k/top_1_categorical_accuracy': 0.014413800090551376,
 'factorized_top_k/top_5_categorical_accuracy': 0.07735690474510193,
 'factorized_top_k/top_10_categorical_accuracy': 0.1351720094680786,
 'factorized_top_k/top_50_categorical_accuracy': 0.3740764856338501,
 'factorized_top_k/top_100_categorical_accuracy': 0.5078839063644409,
 'loss': 9417.044921875,
 'regularization_loss': 0,
 'total_loss': 9417.044921875}

- factorized_top_k/top_1_categorical_accuracy : 모델이 가장 높은 점수를 준 추천이 실제로 사용자가 선택한 항목과 일치하는 비율입니다. 여기서는 약 0.0167, 즉 약 1.67%입니다.
- factorized_top_k/top_5_categorical_accuracy : 모델이 제공한 상위 5개의 추천 중 실제 사용자가 선택한 항목이 포함되는 비율입니다. 여기서는 약 0.0828, 즉 약 8.28%입니다.
- factorized_top_k/top_10_categorical_accuracy : 모델이 제공한 상위 10개의 추천 중 실제 사용자가 선택한 항목이 포함되는 비율입니다. 여기서는 약 0.1413, 즉 약 14.13%입니다.
- factorized_top_k/top_50_categorical_accuracy : 모델이 제공한 상위 50개의 추천 중 실제 사용자가 선택한 항목이 포함되는 비율입니다. 여기서는 약 0.3801, 즉 약 38.01%입니다.
- factorized_top_k/top_100_categorical_accuracy : 모델이 제공한 상위 100개의 추천 중 실제 사용자가 선택한 항목이 포함되는 비율입니다. 여기서는 약 0.5183, 즉 약 51.83%입니다.

In [13]:
# train_ds에서 하나의 샘플을 선택
sample = next(iter(train_ds.take(1)))

# 샘플의 `context_movie_id`를 이용하여 예측
context_movie_id = sample['context_movie_id']
user_embedding = model._query_model(np.array([context_movie_id]))

# 모든 영화의 임베딩을 계산합니다.
candidate_embeddings = model._candidate_model(unique_movie_ids)

# 사용자 임베딩과 영화 임베딩 간의 유사성을 계산
similarity_scores = tf.matmul(user_embedding, candidate_embeddings, transpose_b=True)

# 가장 높은 점수를 가진 인덱스를 찾습니다.
top_movie_indices = tf.math.top_k(similarity_scores, k=5).indices.numpy()

# 해당 인덱스를 이용해 추천 영화 ID를 찾습니다.
recommended_movie_ids = unique_movie_ids.numpy()[top_movie_indices]

print("Recommended movie IDs for the context movie id", context_movie_id, "are", recommended_movie_ids)

Recommended movie IDs for the context movie id tf.Tensor([b'372' b'141' b'11' b'2706' b'2699' b'2723' b'1441' b'785' b'236' b'339'], shape=(10,), dtype=string) are [[b'2581' b'252' b'1513' b'339' b'236']]


내적 값이 클수록 두 벡터(여기서는 사용자 임베딩과 영화 임베딩) 사이의 유사성이 더 크다는 것을 의미